# This notebook is used to generate scene-level learning jobs on Victorian On-bike Cycling (legacy) dataset

In [1]:
import os
import pandas as pd
import numpy as np
import subprocess
from tqdm import tqdm
import time
import shutil
import cv2

In [2]:
csv_data_path = r"/project/6055479/lingheng/cyc_near_miss/dataset_utils/dataset_vic_cyc_legacy/NearMiss_classification.csv"
rgb_frame_data_path = r"/scratch/lingheng/cyc_safety_project/vic_cyc_legacy_pass_event_clip_rgb_frame/"
opt_frame_data_path = r"/scratch/lingheng/cyc_safety_project/vic_cyc_legacy_pass_event_clip_opt_frame/"
fused_frame_data_path = r"/scratch/lingheng/cyc_safety_project/vic_cyc_legacy_pass_event_clip_fused_frame/"


## CNN+LSTM

In [40]:
job_script_root_dir = '/scratch/lingheng/cyc_safety_job_crnn_orig_scripts'
job_output_root_dir = '/scratch/lingheng/cyc_safety_job_crnn_orig_output'
if not os.path.exists(job_script_root_dir):
    os.makedirs(job_script_root_dir)
else:
    # If exist, remove to avoid recreate job scripts
    shutil.rmtree(job_script_root_dir)
    os.makedirs(job_script_root_dir)
    
if not os.path.exists(job_output_root_dir):
    os.makedirs(job_output_root_dir)

In [41]:
run_num = 5
input_type = 'rgb_only' # 'rgb_only', 'fused', 'rgb_and_opt'
exp_data_dir = r"/scratch/lingheng/cyc_safety_project_results/New_CNN_LSTM_{}_input".format(input_type)

for run_i in range(run_num):
    # Get start and end id for the batch
    print('run_i={}'.format(run_i))
    #
    job_script_name = 'job_crnn_orig-{}-{}.sh'.format(input_type, run_i)
    job_output_name = 'output_crnn_orig-{}-{}_R-%x.%j.out'.format(input_type, run_i)
    # Write batch job script
    with open(os.path.join(job_script_root_dir, job_script_name), 'w', encoding='utf8', newline='\n') as f:
        f.write('''\
#!/bin/bash
#SBATCH --account=def-rgorbet
#SBATCH --nodes=1                # When requesting GPUs per node the number of nodes must be specified.
#SBATCH --gpus-per-node=1
#SBATCH --ntasks=5               # number of MPI processes
#SBATCH --mem-per-cpu=10000M          # memory; default unit is megabytes
#SBATCH --time=0-20:0         # time (DD-HH:MM)
#SBATCH --output={0}/{1}

source /home/lingheng/cyc_env/bin/activate
module load python/3.8
pip3 install ipython
python /project/6055479/lingheng/cyc_near_miss/cnm/scene_lvl/crnn_orig/crnn.py --csv_data_path {2} --rgb_frame_data_path {3} --opt_frame_data_path {4} --fused_frame_data_path {5} --input_type {6} --epochs 50 --exp_name {7} --exp_data_dir {8}
                '''.format(job_output_root_dir, job_output_name,
                           csv_data_path, rgb_frame_data_path, opt_frame_data_path, fused_frame_data_path, input_type,
                           'crnn_orig_run_{}'.format(run_i), 
                            exp_data_dir))

run_i=0
run_i=1
run_i=2
run_i=3
run_i=4


In [42]:
job_script_list = os.listdir(job_script_root_dir)
job_script_list.sort()
job_script_list

['job_crnn_orig-rgb_only-0.sh',
 'job_crnn_orig-rgb_only-1.sh',
 'job_crnn_orig-rgb_only-2.sh',
 'job_crnn_orig-rgb_only-3.sh',
 'job_crnn_orig-rgb_only-4.sh']

In [43]:
len(job_script_list)

5

In [44]:
for job_i, job_script in enumerate(job_script_list):
    job_script_path = os.path.join(job_script_root_dir, job_script)
    print('Submitting {}: {}'.format(job_i, job_script_path))  
    !sbatch {job_script_path} 
    print("#########################################################################################")

Submitting 0: /scratch/lingheng/cyc_safety_job_crnn_orig_scripts/job_crnn_orig-rgb_only-0.sh
Submitted batch job 63489470
#########################################################################################
Submitting 1: /scratch/lingheng/cyc_safety_job_crnn_orig_scripts/job_crnn_orig-rgb_only-1.sh
Submitted batch job 63489471
#########################################################################################
Submitting 2: /scratch/lingheng/cyc_safety_job_crnn_orig_scripts/job_crnn_orig-rgb_only-2.sh
Submitted batch job 63489472
#########################################################################################
Submitting 3: /scratch/lingheng/cyc_safety_job_crnn_orig_scripts/job_crnn_orig-rgb_only-3.sh
Submitted batch job 63489473
#########################################################################################
Submitting 4: /scratch/lingheng/cyc_safety_job_crnn_orig_scripts/job_crnn_orig-rgb_only-4.sh
Submitted batch job 63489474
##############################

## I3D

In [17]:
job_script_root_dir = '/scratch/lingheng/cyc_safety_job_scripts_i3d'
job_output_root_dir = '/scratch/lingheng/cyc_safety_job_output_i3d'
if not os.path.exists(job_script_root_dir):
    os.makedirs(job_script_root_dir)
else:
    # If exist, remove to avoid recreate job scripts
    shutil.rmtree(job_script_root_dir)
    os.makedirs(job_script_root_dir)
    
if not os.path.exists(job_output_root_dir):
    os.makedirs(job_output_root_dir)

In [18]:
run_num = 5
input_type = 'rgb_and_opt' # 'rgb_only', 'fused', 'rgb_and_opt'
exp_data_dir = r"/scratch/lingheng/cyc_safety_project_results/New_i3d_WithDropout_{}_input".format(input_type)

for run_i in range(run_num):
    # Get start and end id for the batch
    print('run_i={}'.format(run_i))
    #
    job_script_name = 'job_i3d-{}-{}.sh'.format(input_type, run_i)
    job_output_name = 'output_i3d-{}-{}_R-%x.%j.out'.format(input_type, run_i)
    # Write batch job script
    with open(os.path.join(job_script_root_dir, job_script_name), 'w', encoding='utf8', newline='\n') as f:
        f.write('''\
#!/bin/bash
#SBATCH --account=def-rgorbet
#SBATCH --nodes=1                # When requesting GPUs per node the number of nodes must be specified.
#SBATCH --gpus-per-node=1
#SBATCH --ntasks=5               # number of MPI processes
#SBATCH --mem-per-cpu=10000M          # memory; default unit is megabytes
#SBATCH --time=0-20:0         # time (DD-HH:MM)
#SBATCH --output={0}/{1}

source /home/lingheng/cyc_env/bin/activate
module load python/3.8
pip3 install ipython
python /project/6055479/lingheng/cyc_near_miss/cnm/scene_lvl/i3d/i3d.py --csv_data_path {2} --rgb_frame_data_path {3} --opt_frame_data_path {4} --fused_frame_data_path {5} --input_type {6} --epochs 50 --dropout_p 0.1 --exp_name {7} --exp_data_dir {8}
                '''.format(job_output_root_dir, job_output_name,
                           csv_data_path, rgb_frame_data_path, opt_frame_data_path, fused_frame_data_path, input_type,
                           'i3d_run_{}'.format(run_i), 
                            exp_data_dir))

run_i=0
run_i=1
run_i=2
run_i=3
run_i=4


In [19]:
job_script_list = os.listdir(job_script_root_dir)
job_script_list.sort()
job_script_list

['job_i3d-rgb_and_opt-0.sh',
 'job_i3d-rgb_and_opt-1.sh',
 'job_i3d-rgb_and_opt-2.sh',
 'job_i3d-rgb_and_opt-3.sh',
 'job_i3d-rgb_and_opt-4.sh']

In [20]:
len(job_script_list)

5

In [21]:
for job_i, job_script in enumerate(job_script_list):
    job_script_path = os.path.join(job_script_root_dir, job_script)
    print('Submitting {}: {}'.format(job_i, job_script_path))  
    !sbatch {job_script_path} 
    print("#########################################################################################")

Submitting 0: /scratch/lingheng/cyc_safety_job_scripts_i3d/job_i3d-rgb_and_opt-0.sh
Submitted batch job 63548085
#########################################################################################
Submitting 1: /scratch/lingheng/cyc_safety_job_scripts_i3d/job_i3d-rgb_and_opt-1.sh
Submitted batch job 63548086
#########################################################################################
Submitting 2: /scratch/lingheng/cyc_safety_job_scripts_i3d/job_i3d-rgb_and_opt-2.sh
Submitted batch job 63548088
#########################################################################################
Submitting 3: /scratch/lingheng/cyc_safety_job_scripts_i3d/job_i3d-rgb_and_opt-3.sh
Submitted batch job 63548089
#########################################################################################
Submitting 4: /scratch/lingheng/cyc_safety_job_scripts_i3d/job_i3d-rgb_and_opt-4.sh
Submitted batch job 63548090
###########################################################################